<a href="https://colab.research.google.com/github/moonryul/A.I.-Jumping-Cars-ML-Agents-Example/blob/master/Copy_of_02_fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import numpy as np

In [ ]:
  from google.colab import drive
  drive.mount('/content/gdrive')

Mounted at /content/gdrive


## The forward and backward passes

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2')


In [ ]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2']

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [ ]:
#export
from exp.nb_01 import *



def get_data():
    import os
    import torchvision.datasets as datasets
    
    datasets.MNIST.resources = [
        ('https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz', 'f68b3c2dcbeaaa9fbdd348bbdeb94873'),
        ('https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz', 'd53e105ee54ea40749a09fcbcd1e9432'),
        ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz', '9fb629c4189551a2d022fa330f9573f3'),
        ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz', 'ec29112dd5afa0611ce80d1b7f02629c')
    ]
    
    root = '../data'
    if not os.path.exists(root):
        os.mkdir(root)
    train_set = datasets.MNIST(root=root, train=True, download=True)
    test_set = datasets.MNIST(root=root, train=False, download=True)
    x_train, x_valid = train_set.data.split([50000, 10000])
    y_train, y_valid = train_set.targets.split([50000, 10000])
    return (x_train.view(50000, -1) / 256.0), y_train.float(), (x_valid.view(10000, -1))/ 256.0, y_valid.float()

#x_train,y_train,x_valid,y_valid = get_data()


#def get_data():
#    path = datasets.download_data(MNIST_URL, ext='.gz')
#    with gzip.open(path, 'rb') as f:
#        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
#    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
len(x_train)

50000

In [ ]:
y_train[0:10]

tensor([5., 0., 4., 1., 9., 2., 1., 3., 1., 4.])

In [ ]:
x_train[0].shape

torch.Size([784])

In [ ]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [ ]:
sizes = [2,3,1]
sizes[1:]

[3, 1]

In [ ]:
biases = [np.random.randn(y, 1) for y in sizes[1:]]

In [ ]:
biases

[array([[ 0.469608],
        [-1.75423 ],
        [-2.03047 ]]), array([[1.918446]])]

In [ ]:
sizes[:-1]

[2, 3]

In [ ]:
iterable = zip( sizes[0:-1], sizes[1:])

In [ ]:
iterable[0]

TypeError: ignored

In [ ]:
[(x,y) for x,y in iterable]

[(2, 3), (3, 1)]

In [ ]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [ ]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(3.8966e-08), tensor(1.))

In [ ]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [ ]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [ ]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10.))

## Foundations version

### Basic architecture

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [ ]:
# num hidden
nh = 50
nh = 5

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

In [ ]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [ ]:
m

784

In [ ]:
nh

5

In [ ]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [ ]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [ ]:
def lin(x, w, b): return x@w + b

In [ ]:
t = lin(x_valid, w1, b1)

In [ ]:
x_valid.shape

torch.Size([10000, 784])

In [ ]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.7129), tensor(0.9236))

In [ ]:
def relu(x): return x.clamp_min(0.)

In [ ]:
t = relu(lin(x_valid, w1, b1))

In [ ]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.2903), tensor(0.4646))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [ ]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [ ]:
w1.mean(),w1.std()

(tensor(0.0017), tensor(0.0514))

In [ ]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.3938), tensor(0.6324))

In [ ]:
#export
from torch.nn import init

In [ ]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [ ]:

w1.shape

torch.Size([784, 5])

In [ ]:
nh

5

In [ ]:
w1.t().shape

torch.Size([5, 784])

In [ ]:
init.kaiming_normal_??

In [ ]:
w1.mean(),w1.std()

(tensor(-0.0004), tensor(0.0513))

In [ ]:
t.mean(),t.std()

(tensor(0.4372), tensor(0.7088))

In [ ]:
w1.shape

torch.Size([784, 50])

In [ ]:
import torch.nn

In [ ]:
x_valid[0].shape

torch.Size([784])

In [ ]:
x_valid[0][:5].t().matmul(w1.t()).shape

torch.Size([784])

In [ ]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [ ]:
torch.nn.Linear.forward??

In [ ]:
torch.nn.functional.linear??

In [ ]:
torch.nn.Conv2d??

In [ ]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [ ]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [ ]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(-0.1095), tensor(0.6229))

In [ ]:
def model(xb):
    l1 = lin(xb, w1, b1)
    #print ("l1=", l1)
    l2 = relu(l1)
    #print ("l2 = ", l2)
    l3 = lin(l2, w2, b2)
    #print ("l3 = ", l3)
    return l3

In [ ]:
x_valid[0].shape

torch.Size([784])

In [ ]:
 l1=model(x_valid[:2])

l1= tensor([[ 1.0630, -1.3090, -1.6371,  0.8821,  1.2616],
        [ 0.7204, -1.0949,  1.1853,  0.5240, -0.2975]])


In [ ]:
l1.shape

torch.Size([2, 1])

In [ ]:
%timeit -n 10 _=model(x_valid[0])

In [ ]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [ ]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

$x_train$ is a set of input vectors to the neural net with two layers: \\
$x\_train \\
= X 
= \begin{bmatrix}  x^{(1)}  \\ \vdots \\ x^{(m)}  \end{bmatrix} = 
\begin{bmatrix} 
 x_{1}^{(1)} & \cdots  & x_{n}^{(1)} \\ 
 \vdots          & \cdots & \vdots \\ 
 x_{1}^{(m)} & \cdots &  x_{n}^{(m)} \end{bmatrix} $

 Here, the index in the parenthesis superfix 1 in $x^{(1)}$ means that input vector $x^{(1)}$ is from  from data sample 1.  The suffix 1 in $x_{1}^{(1)}$ means that $z_{1}^{(1)}$  is the first element of vector $z^{(1)}$. 
 In summary, note that the input to the neural net for training is represented as a matrix X. In this representation, the rows of the matrix X refer to samples. Some software and papers represent X so that the columns of X refer to samples.
 
 $output$ is the output vector of the neural net with two layers: \\
$output \\
= Z^{[2]}
= \begin{bmatrix}  z^{[2](1)}  \\ \vdots \\ z^{[2](n)}  \end{bmatrix} = 
\begin{bmatrix} 
 z_{1}^{[2](1)} & \cdots  & z_{m^{[2]}}^{[2](1)} \\ 
 \vdots          & \cdots & \vdots \\ 
 z_{1}^{[2](n)} & \cdots &  z_{m^{[2]}}^{[2](n)} \end{bmatrix} $

 Here, the index in the square superfix 2 in $z^{[2](1)}$ means that the variable $z^{[2](1)}$ lives on the 2nd layer of the network. The 0th layer of the network refers to the input layer. 
 The index in the parenthesis superfix 1 in $z^{[2](1)}$ means that the variable $z^{[2](1)}$ is computed from data sample 1.  The suffix 1 in $z_{1}^{[2](1)}$ means that $z_{1}^{[2](1)}$  is the first element of vector $z{[2](1)}$. There are $m^{[2]}$ elements in the output vector $z^{[2](i)}$ on the 2nd layer.

$ Z^{[2]}
=  X W^{[2]} + b^{[2]}$
 $\begin{bmatrix} 
 z_{1}^{[2](1)} & \cdots  & z_{m^{[2]}}^{[2](1)} \\ 
 \vdots          & \cdots & \vdots \\ 
 z_{1}^{[2](n)} & \cdots &  z_{m^{[2]}}^{[2](n)} \end{bmatrix} = 
\begin{bmatrix} 
 x_{1}^{(1)} & \cdots  & x_{n}^{(1)} \\ 
 \vdots          & \cdots & \vdots \\ 
 x_{1}^{(m)} & \cdots &  x_{n}^{(m)} \end{bmatrix}
 
\begin{bmatrix} 
 w_{11}^{[2]} & \cdots  & w_{1m^{[2]}}^{[2]} \\ 
 \vdots          & \cdots & \vdots \\ 
 w_{n1}^{[2]} & \cdots &  w_{nm^{[2]}}^{[2]} \end{bmatrix} 
 +  
  \begin{bmatrix}  b_{1}^{[2]}  \\ \vdots \\ b_{n}^{[2]}  \end{bmatrix} $

In [ ]:
#export
def mse(output, y_train): return (output.squeeze(-1) - y_train).pow(2).mean()

$$\begin{bmatrix}  z^{[2](1)}  \\ z^{[2](2)}  \end{bmatrix}$$

In [ ]:
y_train,y_valid = y_train.float(),y_valid.float()

In [ ]:
preds = model(x_train)

In [ ]:
preds.shape

torch.Size([50000, 1])

In [ ]:
mse(preds, y_train)

tensor(48.3247)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [ ]:
def mse_grad(inp, targ): # inp =x_train, targ = y_train
    # grad of loss with respect to output of previous layer
    # print (inp.g)

    #print (inp.g.shape)
    # tensor x_train is given a new field that contains its gradient relative y_hat
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]
    print (inp.g.shape)

In [ ]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [ ]:
def lin_grad(inp, out, w, b):# d out/ dw^{2}, inp = a^{1}, out = y_hat, b=b^{2}
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [ ]:
def forward_and_backward(x_train, y_train): 
    # forward pass:
    z1 = x_train @ w1 + b1 # z1 is a matrix
    a1 = relu(z1)          # a1 is a matrix
    z2 = a1 @ w2 + b2      # z2 is a matrix

 
    # we don't actually need the loss in backward!
    loss = mse(z2, y_train) 

    # backward pass:
    mse_grad(z2, y_train) 

    #print (out.g.shape)
    lin_grad(a1, z2, w2, b2) # xf. z2 = a1 @ w2 + b2
    relu_grad(z1, a1)        # cf. a1 = relu(z1)
    lin_grad(x_train, z1, w1, b1) #cf. z1 = x_train @ w1 + b1

In [ ]:
forward_and_backward(x_train, y_train)

torch.Size([50000, 1])
torch.Size([50000, 1])


In [ ]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [ ]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [ ]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [ ]:
loss = forward(xt2, y_train)

In [ ]:
loss.backward()

In [ ]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [ ]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 137 ms, sys: 4.95 ms, total: 142 ms
Wall time: 70.7 ms


In [ ]:
%time model.backward()

CPU times: user 2.84 s, sys: 3.86 s, total: 6.71 s
Wall time: 3.4 s


In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [ ]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 86 ms, sys: 8.25 ms, total: 94.2 ms
Wall time: 46.3 ms


In [ ]:
%time model.backward()

CPU times: user 193 ms, sys: 87.6 ms, total: 280 ms
Wall time: 140 ms


In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [ ]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 88.6 ms, sys: 5.04 ms, total: 93.6 ms
Wall time: 46.4 ms


In [ ]:
%time model.backward()

CPU times: user 197 ms, sys: 83.9 ms, total: 281 ms
Wall time: 140 ms


In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [ ]:
#export
from torch import nn

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [ ]:
model = Model(m, nh, 1)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 85.1 ms, sys: 8.16 ms, total: 93.3 ms
Wall time: 46.3 ms


In [ ]:
%time loss.backward()

CPU times: user 135 ms, sys: 78.1 ms, total: 213 ms
Wall time: 71.1 ms


## Export

In [ ]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to nb_02.py
